In [1]:
D.<x> =PolynomialRing(QQ)
DD=PolynomialRing(QQ,'q')
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]

def Delta(n):
    M = ModularForms(SL2Z, 12) 
    return DD(M.q_expansion_basis(n)[0])

def tau(n):
    return Delta(n+1)[n]
 
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})


print("okay")

okay


In [70]:
ls=[]
for n in [1..5]:ls=ls+[tau(n)]
print(ls)

[1, -24, 252, -1472, 4830]


In [71]:
def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)

print(RamanujanCongruenceSeries(5))



-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x


In [75]:
def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)


def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]

for k in [1..3]:
    print(RamanujanCongruenceSeries(n));print()
    print(recip(n)^k);print()
    rpc=recipPowerConstantTerm(k)
    test=order(2,rpc)-3*digitsum(2,k)
    print((rpc,test))
    
    print("----------------------------------------------------------")

-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-1 + 24 + 324*x + 3200*x^2 + 25650*x^3 + 176256*x^4 + 1056976*x^5 + O(x^6)

(24, 0)
----------------------------------------------------------
-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-2 + 48*x^-1 + 1224 + 21952*x + 309876*x^2 + 3657312*x^3 + 37435440*x^4 + O(x^5)

(1224, 0)
----------------------------------------------------------
-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

x^-3 + 72*x^-2 + 2700*x^-1 + 70080 + 1412550*x + 23531040*x^2 + 336770016*x^3 + O(x^4)

(70080, 0)
----------------------------------------------------------


In [2]:
import random
import time


def specialModulus(a):
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    return 6*8^ds2*3^ds3

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-100, 100)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
no=[]
for b in [1..10]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        o3=order(3,c)
        sk=c/3^o3
        m=mod(sk,3)
        if ((mod(a,2)==1)&(m!=2)):no=no+[a]
        if ((mod(a,2)==0)&(m!=1)):no=no+[a]
        ds2=digitsum(2,a)
        ds3=digitsum(3,a)
        test2=o2-3*ds2
        test3=o3-ds3
        print((b,a,m,len(no),test2,test3))
        tests=tests+[(test2,test3)]
    print();print("trial",b)
    print();print("tests:");print(tests)
    print();print("len(no):");print(len(no))
    print("-----------------------------------------------------------------------")


(1, 1, 2, 0, 0, 0)
(1, 2, 1, 0, 0, 0)
(1, 3, 2, 0, 0, 0)
(1, 4, 1, 0, 0, 0)
(1, 5, 2, 0, 0, 0)
(1, 6, 1, 0, 0, 0)
(1, 7, 2, 0, 0, 0)
(1, 8, 1, 0, 0, 0)
(1, 9, 2, 0, 0, 0)
(1, 10, 1, 0, 0, 0)
(1, 11, 2, 0, 0, 0)
(1, 12, 1, 0, 0, 0)
(1, 13, 2, 0, 0, 0)
(1, 14, 1, 0, 0, 0)
(1, 15, 2, 0, 0, 0)
(1, 16, 1, 0, 0, 0)
(1, 17, 2, 0, 0, 0)
(1, 18, 1, 0, 0, 0)
(1, 19, 2, 0, 0, 0)
(1, 20, 1, 0, 0, 0)
(1, 21, 2, 0, 0, 0)
(1, 22, 1, 0, 0, 0)
(1, 23, 2, 0, 0, 0)
(1, 24, 1, 0, 0, 0)
(1, 25, 2, 0, 0, 0)
(1, 26, 1, 0, 0, 0)
(1, 27, 2, 0, 0, 0)
(1, 28, 1, 0, 0, 0)
(1, 29, 2, 0, 0, 0)
(1, 30, 1, 0, 0, 0)
(1, 31, 2, 0, 0, 0)
(1, 32, 1, 0, 0, 0)
(1, 33, 2, 0, 0, 0)
(1, 34, 1, 0, 0, 0)
(1, 35, 2, 0, 0, 0)
(1, 36, 1, 0, 0, 0)
(1, 37, 2, 0, 0, 0)
(1, 38, 1, 0, 0, 0)
(1, 39, 2, 0, 0, 0)
(1, 40, 1, 0, 0, 0)
(1, 41, 2, 0, 0, 0)
(1, 42, 1, 0, 0, 0)
(1, 43, 2, 0, 0, 0)
(1, 44, 1, 0, 0, 0)
(1, 45, 2, 0, 0, 0)
(1, 46, 1, 0, 0, 0)
(1, 47, 2, 0, 0, 0)
(1, 48, 1, 0, 0, 0)
(1, 49, 2, 0, 0, 0)
(1, 50, 1, 0, 0, 0)
(1, 51, 

(1, 397, 2, 0, 0, 0)
(1, 398, 1, 0, 0, 0)
(1, 399, 2, 0, 0, 0)
(1, 400, 1, 0, 0, 0)
(1, 401, 2, 0, 0, 0)
(1, 402, 1, 0, 0, 0)
(1, 403, 2, 0, 0, 0)
(1, 404, 1, 0, 0, 0)
(1, 405, 2, 0, 0, 0)
(1, 406, 1, 0, 0, 0)
(1, 407, 2, 0, 0, 0)
(1, 408, 1, 0, 0, 0)
(1, 409, 2, 0, 0, 0)
(1, 410, 1, 0, 0, 0)
(1, 411, 2, 0, 0, 0)
(1, 412, 1, 0, 0, 0)
(1, 413, 2, 0, 0, 0)
(1, 414, 1, 0, 0, 0)
(1, 415, 2, 0, 0, 0)
(1, 416, 1, 0, 0, 0)
(1, 417, 2, 0, 0, 0)
(1, 418, 1, 0, 0, 0)
(1, 419, 2, 0, 0, 0)
(1, 420, 1, 0, 0, 0)
(1, 421, 2, 0, 0, 0)
(1, 422, 1, 0, 0, 0)
(1, 423, 2, 0, 0, 0)
(1, 424, 1, 0, 0, 0)
(1, 425, 2, 0, 0, 0)
(1, 426, 1, 0, 0, 0)
(1, 427, 2, 0, 0, 0)
(1, 428, 1, 0, 0, 0)
(1, 429, 2, 0, 0, 0)
(1, 430, 1, 0, 0, 0)
(1, 431, 2, 0, 0, 0)
(1, 432, 1, 0, 0, 0)
(1, 433, 2, 0, 0, 0)
(1, 434, 1, 0, 0, 0)
(1, 435, 2, 0, 0, 0)
(1, 436, 1, 0, 0, 0)
(1, 437, 2, 0, 0, 0)
(1, 438, 1, 0, 0, 0)
(1, 439, 2, 0, 0, 0)
(1, 440, 1, 0, 0, 0)
(1, 441, 2, 0, 0, 0)
(1, 442, 1, 0, 0, 0)
(1, 443, 2, 0, 0, 0)
(1, 444, 1, 0

(2, 97, 2, 0, 0, 0)
(2, 98, 1, 0, 0, 0)
(2, 99, 2, 0, 0, 0)
(2, 100, 1, 0, 0, 0)
(2, 101, 2, 0, 0, 0)
(2, 102, 1, 0, 0, 0)
(2, 103, 2, 0, 0, 0)
(2, 104, 1, 0, 0, 0)
(2, 105, 2, 0, 0, 0)
(2, 106, 1, 0, 0, 0)
(2, 107, 2, 0, 0, 0)
(2, 108, 1, 0, 0, 0)
(2, 109, 2, 0, 0, 0)
(2, 110, 1, 0, 0, 0)
(2, 111, 2, 0, 0, 0)
(2, 112, 1, 0, 0, 0)
(2, 113, 2, 0, 0, 0)
(2, 114, 1, 0, 0, 0)
(2, 115, 2, 0, 0, 0)
(2, 116, 1, 0, 0, 0)
(2, 117, 2, 0, 0, 0)
(2, 118, 1, 0, 0, 0)
(2, 119, 2, 0, 0, 0)
(2, 120, 1, 0, 0, 0)
(2, 121, 2, 0, 0, 0)
(2, 122, 1, 0, 0, 0)
(2, 123, 2, 0, 0, 0)
(2, 124, 1, 0, 0, 0)
(2, 125, 2, 0, 0, 0)
(2, 126, 1, 0, 0, 0)
(2, 127, 2, 0, 0, 0)
(2, 128, 1, 0, 0, 0)
(2, 129, 2, 0, 0, 0)
(2, 130, 1, 0, 0, 0)
(2, 131, 2, 0, 0, 0)
(2, 132, 1, 0, 0, 0)
(2, 133, 2, 0, 0, 0)
(2, 134, 1, 0, 0, 0)
(2, 135, 2, 0, 0, 0)
(2, 136, 1, 0, 0, 0)
(2, 137, 2, 0, 0, 0)
(2, 138, 1, 0, 0, 0)
(2, 139, 2, 0, 0, 0)
(2, 140, 1, 0, 0, 0)
(2, 141, 2, 0, 0, 0)
(2, 142, 1, 0, 0, 0)
(2, 143, 2, 0, 0, 0)
(2, 144, 1, 0, 0

(2, 488, 1, 0, 0, 0)
(2, 489, 2, 0, 0, 0)
(2, 490, 1, 0, 0, 0)
(2, 491, 2, 0, 0, 0)
(2, 492, 1, 0, 0, 0)
(2, 493, 2, 0, 0, 0)
(2, 494, 1, 0, 0, 0)
(2, 495, 2, 0, 0, 0)
(2, 496, 1, 0, 0, 0)
(2, 497, 2, 0, 0, 0)
(2, 498, 1, 0, 0, 0)
(2, 499, 2, 0, 0, 0)
(2, 500, 1, 0, 0, 0)

trial 2

tests:
[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0

(3, 3, 2, 0, 0, 0)
(3, 4, 1, 0, 0, 0)
(3, 5, 2, 0, 0, 0)
(3, 6, 1, 0, 0, 0)
(3, 7, 2, 0, 0, 0)
(3, 8, 1, 0, 0, 0)
(3, 9, 2, 0, 0, 0)
(3, 10, 1, 0, 0, 0)
(3, 11, 2, 0, 0, 0)
(3, 12, 1, 0, 0, 0)
(3, 13, 2, 0, 0, 0)
(3, 14, 1, 0, 0, 0)
(3, 15, 2, 0, 0, 0)
(3, 16, 1, 0, 0, 0)
(3, 17, 2, 0, 0, 0)
(3, 18, 1, 0, 0, 0)
(3, 19, 2, 0, 0, 0)
(3, 20, 1, 0, 0, 0)
(3, 21, 2, 0, 0, 0)
(3, 22, 1, 0, 0, 0)
(3, 23, 2, 0, 0, 0)
(3, 24, 1, 0, 0, 0)
(3, 25, 2, 0, 0, 0)
(3, 26, 1, 0, 0, 0)
(3, 27, 2, 0, 0, 0)
(3, 28, 1, 0, 0, 0)
(3, 29, 2, 0, 0, 0)
(3, 30, 1, 0, 0, 0)
(3, 31, 2, 0, 0, 0)
(3, 32, 1, 0, 0, 0)
(3, 33, 2, 0, 0, 0)
(3, 34, 1, 0, 0, 0)
(3, 35, 2, 0, 0, 0)
(3, 36, 1, 0, 0, 0)
(3, 37, 2, 0, 0, 0)
(3, 38, 1, 0, 0, 0)
(3, 39, 2, 0, 0, 0)
(3, 40, 1, 0, 0, 0)
(3, 41, 2, 0, 0, 0)
(3, 42, 1, 0, 0, 0)
(3, 43, 2, 0, 0, 0)
(3, 44, 1, 0, 0, 0)
(3, 45, 2, 0, 0, 0)
(3, 46, 1, 0, 0, 0)
(3, 47, 2, 0, 0, 0)
(3, 48, 1, 0, 0, 0)
(3, 49, 2, 0, 0, 0)
(3, 50, 1, 0, 0, 0)
(3, 51, 2, 0, 0, 0)
(3, 52, 1, 0, 0, 0)
(3, 53,

...

(8, 398, 1, 0, 0, 0)
(8, 399, 2, 0, 0, 0)
(8, 400, 1, 0, 0, 0)
(8, 401, 2, 0, 0, 0)
(8, 402, 1, 0, 0, 0)
(8, 403, 2, 0, 0, 0)
(8, 404, 1, 0, 0, 0)
(8, 405, 2, 0, 0, 0)
(8, 406, 1, 0, 0, 0)
(8, 407, 2, 0, 0, 0)
(8, 408, 1, 0, 0, 0)
(8, 409, 2, 0, 0, 0)
(8, 410, 1, 0, 0, 0)
(8, 411, 2, 0, 0, 0)
(8, 412, 1, 0, 0, 0)
(8, 413, 2, 0, 0, 0)
(8, 414, 1, 0, 0, 0)
(8, 415, 2, 0, 0, 0)
(8, 416, 1, 0, 0, 0)
(8, 417, 2, 0, 0, 0)
(8, 418, 1, 0, 0, 0)
(8, 419, 2, 0, 0, 0)
(8, 420, 1, 0, 0, 0)
(8, 421, 2, 0, 0, 0)
(8, 422, 1, 0, 0, 0)
(8, 423, 2, 0, 0, 0)
(8, 424, 1, 0, 0, 0)
(8, 425, 2, 0, 0, 0)
(8, 426, 1, 0, 0, 0)
(8, 427, 2, 0, 0, 0)
(8, 428, 1, 0, 0, 0)
(8, 429, 2, 0, 0, 0)
(8, 430, 1, 0, 0, 0)
(8, 431, 2, 0, 0, 0)
(8, 432, 1, 0, 0, 0)
(8, 433, 2, 0, 0, 0)
(8, 434, 1, 0, 0, 0)
(8, 435, 2, 0, 0, 0)
(8, 436, 1, 0, 0, 0)
(8, 437, 2, 0, 0, 0)
(8, 438, 1, 0, 0, 0)
(8, 439, 2, 0, 0, 0)
(8, 440, 1, 0, 0, 0)
(8, 441, 2, 0, 0, 0)
(8, 442, 1, 0, 0, 0)
(8, 443, 2, 0, 0, 0)
(8, 444, 1, 0, 0, 0)
(8, 445, 2, 0

(9, 2, 1, 0, 0, 0)
(9, 3, 2, 0, 0, 0)
(9, 4, 1, 0, 0, 0)
(9, 5, 2, 0, 0, 0)
(9, 6, 1, 0, 0, 0)
(9, 7, 2, 0, 0, 0)
(9, 8, 1, 0, 0, 0)
(9, 9, 2, 0, 0, 0)
(9, 10, 1, 0, 0, 0)
(9, 11, 2, 0, 0, 0)
(9, 12, 1, 0, 0, 0)
(9, 13, 2, 0, 0, 0)
(9, 14, 1, 0, 0, 0)
(9, 15, 2, 0, 0, 0)
(9, 16, 1, 0, 0, 0)
(9, 17, 2, 0, 0, 0)
(9, 18, 1, 0, 0, 0)
(9, 19, 2, 0, 0, 0)
(9, 20, 1, 0, 0, 0)
(9, 21, 2, 0, 0, 0)
(9, 22, 1, 0, 0, 0)
(9, 23, 2, 0, 0, 0)
(9, 24, 1, 0, 0, 0)
(9, 25, 2, 0, 0, 0)
(9, 26, 1, 0, 0, 0)
(9, 27, 2, 0, 0, 0)
(9, 28, 1, 0, 0, 0)
(9, 29, 2, 0, 0, 0)
(9, 30, 1, 0, 0, 0)
(9, 31, 2, 0, 0, 0)
(9, 32, 1, 0, 0, 0)
(9, 33, 2, 0, 0, 0)
(9, 34, 1, 0, 0, 0)
(9, 35, 2, 0, 0, 0)
(9, 36, 1, 0, 0, 0)
(9, 37, 2, 0, 0, 0)
(9, 38, 1, 0, 0, 0)
(9, 39, 2, 0, 0, 0)
(9, 40, 1, 0, 0, 0)
(9, 41, 2, 0, 0, 0)
(9, 42, 1, 0, 0, 0)
(9, 43, 2, 0, 0, 0)
(9, 44, 1, 0, 0, 0)
(9, 45, 2, 0, 0, 0)
(9, 46, 1, 0, 0, 0)
(9, 47, 2, 0, 0, 0)
(9, 48, 1, 0, 0, 0)
(9, 49, 2, 0, 0, 0)
(9, 50, 1, 0, 0, 0)
(9, 51, 2, 0, 0, 0)
(9, 52, 

(9, 398, 1, 0, 0, 0)
(9, 399, 2, 0, 0, 0)
(9, 400, 1, 0, 0, 0)
(9, 401, 2, 0, 0, 0)
(9, 402, 1, 0, 0, 0)
(9, 403, 2, 0, 0, 0)
(9, 404, 1, 0, 0, 0)
(9, 405, 2, 0, 0, 0)
(9, 406, 1, 0, 0, 0)
(9, 407, 2, 0, 0, 0)
(9, 408, 1, 0, 0, 0)
(9, 409, 2, 0, 0, 0)
(9, 410, 1, 0, 0, 0)
(9, 411, 2, 0, 0, 0)
(9, 412, 1, 0, 0, 0)
(9, 413, 2, 0, 0, 0)
(9, 414, 1, 0, 0, 0)
(9, 415, 2, 0, 0, 0)
(9, 416, 1, 0, 0, 0)
(9, 417, 2, 0, 0, 0)
(9, 418, 1, 0, 0, 0)
(9, 419, 2, 0, 0, 0)
(9, 420, 1, 0, 0, 0)
(9, 421, 2, 0, 0, 0)
(9, 422, 1, 0, 0, 0)
(9, 423, 2, 0, 0, 0)
(9, 424, 1, 0, 0, 0)
(9, 425, 2, 0, 0, 0)
(9, 426, 1, 0, 0, 0)
(9, 427, 2, 0, 0, 0)
(9, 428, 1, 0, 0, 0)
(9, 429, 2, 0, 0, 0)
(9, 430, 1, 0, 0, 0)
(9, 431, 2, 0, 0, 0)
(9, 432, 1, 0, 0, 0)
(9, 433, 2, 0, 0, 0)
(9, 434, 1, 0, 0, 0)
(9, 435, 2, 0, 0, 0)
(9, 436, 1, 0, 0, 0)
(9, 437, 2, 0, 0, 0)
(9, 438, 1, 0, 0, 0)
(9, 439, 2, 0, 0, 0)
(9, 440, 1, 0, 0, 0)
(9, 441, 2, 0, 0, 0)
(9, 442, 1, 0, 0, 0)
(9, 443, 2, 0, 0, 0)
(9, 444, 1, 0, 0, 0)
(9, 445, 2, 0

(10, 2, 1, 0, 0, 0)
(10, 3, 2, 0, 0, 0)
(10, 4, 1, 0, 0, 0)
(10, 5, 2, 0, 0, 0)
(10, 6, 1, 0, 0, 0)
(10, 7, 2, 0, 0, 0)
(10, 8, 1, 0, 0, 0)
(10, 9, 2, 0, 0, 0)
(10, 10, 1, 0, 0, 0)
(10, 11, 2, 0, 0, 0)
(10, 12, 1, 0, 0, 0)
(10, 13, 2, 0, 0, 0)
(10, 14, 1, 0, 0, 0)
(10, 15, 2, 0, 0, 0)
(10, 16, 1, 0, 0, 0)
(10, 17, 2, 0, 0, 0)
(10, 18, 1, 0, 0, 0)
(10, 19, 2, 0, 0, 0)
(10, 20, 1, 0, 0, 0)
(10, 21, 2, 0, 0, 0)
(10, 22, 1, 0, 0, 0)
(10, 23, 2, 0, 0, 0)
(10, 24, 1, 0, 0, 0)
(10, 25, 2, 0, 0, 0)
(10, 26, 1, 0, 0, 0)
(10, 27, 2, 0, 0, 0)
(10, 28, 1, 0, 0, 0)
(10, 29, 2, 0, 0, 0)
(10, 30, 1, 0, 0, 0)
(10, 31, 2, 0, 0, 0)
(10, 32, 1, 0, 0, 0)
(10, 33, 2, 0, 0, 0)
(10, 34, 1, 0, 0, 0)
(10, 35, 2, 0, 0, 0)
(10, 36, 1, 0, 0, 0)
(10, 37, 2, 0, 0, 0)
(10, 38, 1, 0, 0, 0)
(10, 39, 2, 0, 0, 0)
(10, 40, 1, 0, 0, 0)
(10, 41, 2, 0, 0, 0)
(10, 42, 1, 0, 0, 0)
(10, 43, 2, 0, 0, 0)
(10, 44, 1, 0, 0, 0)
(10, 45, 2, 0, 0, 0)
(10, 46, 1, 0, 0, 0)
(10, 47, 2, 0, 0, 0)
(10, 48, 1, 0, 0, 0)
(10, 49, 2, 0, 0, 0)


(10, 380, 1, 0, 0, 0)
(10, 381, 2, 0, 0, 0)
(10, 382, 1, 0, 0, 0)
(10, 383, 2, 0, 0, 0)
(10, 384, 1, 0, 0, 0)
(10, 385, 2, 0, 0, 0)
(10, 386, 1, 0, 0, 0)
(10, 387, 2, 0, 0, 0)
(10, 388, 1, 0, 0, 0)
(10, 389, 2, 0, 0, 0)
(10, 390, 1, 0, 0, 0)
(10, 391, 2, 0, 0, 0)
(10, 392, 1, 0, 0, 0)
(10, 393, 2, 0, 0, 0)
(10, 394, 1, 0, 0, 0)
(10, 395, 2, 0, 0, 0)
(10, 396, 1, 0, 0, 0)
(10, 397, 2, 0, 0, 0)
(10, 398, 1, 0, 0, 0)
(10, 399, 2, 0, 0, 0)
(10, 400, 1, 0, 0, 0)
(10, 401, 2, 0, 0, 0)
(10, 402, 1, 0, 0, 0)
(10, 403, 2, 0, 0, 0)
(10, 404, 1, 0, 0, 0)
(10, 405, 2, 0, 0, 0)
(10, 406, 1, 0, 0, 0)
(10, 407, 2, 0, 0, 0)
(10, 408, 1, 0, 0, 0)
(10, 409, 2, 0, 0, 0)
(10, 410, 1, 0, 0, 0)
(10, 411, 2, 0, 0, 0)
(10, 412, 1, 0, 0, 0)
(10, 413, 2, 0, 0, 0)
(10, 414, 1, 0, 0, 0)
(10, 415, 2, 0, 0, 0)
(10, 416, 1, 0, 0, 0)
(10, 417, 2, 0, 0, 0)
(10, 418, 1, 0, 0, 0)
(10, 419, 2, 0, 0, 0)
(10, 420, 1, 0, 0, 0)
(10, 421, 2, 0, 0, 0)
(10, 422, 1, 0, 0, 0)
(10, 423, 2, 0, 0, 0)
(10, 424, 1, 0, 0, 0)
(10, 425, 